<a href="https://colab.research.google.com/github/Fakhryrama/Data_Mining/blob/main/Data_Stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_hum = pd.read_csv('/content/drive/MyDrive/Datasets/humidity.csv')
df_hum.head()

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN
1,2012-10-01 13:00:00,76.0,81.0,88.0,81.0,88.0,82.0,22.0,23.0,50.0,...,71.0,58.0,93.0,68.0,50.0,63.0,22.0,51.0,51.0,50.0
2,2012-10-01 14:00:00,76.0,80.0,87.0,80.0,88.0,81.0,21.0,23.0,49.0,...,70.0,57.0,91.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0
3,2012-10-01 15:00:00,76.0,80.0,86.0,80.0,88.0,81.0,21.0,23.0,49.0,...,70.0,57.0,87.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0
4,2012-10-01 16:00:00,77.0,80.0,85.0,79.0,88.0,81.0,21.0,23.0,49.0,...,69.0,57.0,84.0,68.0,52.0,62.0,22.0,51.0,51.0,50.0


# Sampling Method(Sliding Window)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import random
import numpy as np
import time

# Check if 'datetime' column exists before dropping
if 'datetime' in df_hum.columns:
    X = df_hum.drop(columns=['datetime']).values
else:
    print("Column 'datetime' not found in DataFrame. Proceeding without dropping it.")
    X = df_hum.values  # Use all columns if 'datetime' is not found

# Parameter Streaming
t_len = 50  # ukuran window
End_id_data_Stream = random.randint(50, len(X))  # minimal 50 agar ada cukup data

# Buffer untuk Sliding Window
buffer_x = []

# Streaming loop
for i in range(End_id_data_Stream):
    if i >= t_len:
        buffer_x = X[i - t_len:i]  # ambil t_len data terakhir
    else:
        buffer_x = X[0:i+1]  # saat awal belum cukup t_len data

    buffer_array = np.array(buffer_x)

    # Contoh pemrosesan: hitung rata-rata tiap kolom
    avg_per_city = np.mean(buffer_array, axis=0)

    print(f"Step {i+1}/{End_id_data_Stream}, Window size: {len(buffer_x)}")
    print(f"Rata-rata kelembaban per kota:\n{avg_per_city}\n")

    time.sleep(0.001)  # jeda 10 ms (bisa disesuaikan)

Streaming output truncated to the last 5000 lines.
 65.82 56.26 67.38 55.68 57.88 51.78 67.32 45.86 42.64 66.54 66.54 43.48]

Step 23547/24379, Window size: 50
Rata-rata kelembaban per kota:
[81.84 67.68 65.1  74.88 76.26 78.8  81.96 50.86 56.98 38.64 67.22 59.36
 65.54 41.74 47.9  45.36 61.2  42.1  64.7  39.34 76.58 37.04 44.34 54.86
 66.   57.22 67.18 55.42 57.24 52.5  68.02 47.08 44.04 67.02 67.02 44.7 ]

Step 23548/24379, Window size: 50
Rata-rata kelembaban per kota:
[81.92 67.76 66.1  75.38 75.7  78.72 81.88 51.64 56.92 39.14 67.98 60.28
 66.48 42.48 47.96 46.06 62.18 42.   65.08 39.38 76.9  37.7  44.6  55.08
 66.06 58.18 67.12 55.36 56.6  53.22 68.72 48.26 45.44 67.5  67.5  45.88]

Step 23549/24379, Window size: 50
Rata-rata kelembaban per kota:
[81.62 68.08 66.78 75.84 74.92 78.46 81.9  52.3  56.44 39.86 68.82 61.2
 67.24 43.22 47.74 46.82 62.86 41.98 65.74 39.64 76.86 38.48 44.7  55.26
 66.1  58.96 67.24 55.26 56.9  53.34 69.34 49.58 45.92 67.82 67.82 47.2 ]

Step 23550/24379,

In [ ]:
buffer_x.shape

(50, 36)

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Pilih target kota
target_column = 'Vancouver'

# Check if columns exist before dropping
columns_to_drop = [target_column]
if 'datetime' in df_hum.columns:
    columns_to_drop.insert(0, 'datetime')

# Fitur = semua kota kecuali target
X = df_hum.drop(columns=columns_to_drop).values
y = df_hum[target_column].values

# Scaling fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

buffer_x = X_scaled
buffer_y = y

if len(buffer_x) >= 10:
    X_train, X_test, y_train, y_test = train_test_split(
        buffer_x, buffer_y, test_size=0.2, random_state=42
    )

    # Model regresi
    model = SGDRegressor(random_state=42, max_iter=1000, tol=1e-3)
    model.fit(X_train, y_train)

    # Prediksi
    y_pred = model.predict(X_test)

    # Evaluasi threshold-based accuracy
    threshold = 5.0
    accurate_predictions = np.where(np.abs(y_test - y_pred) <= threshold, 1, 0)
    accuracy = np.mean(accurate_predictions)

    print(f"Akurasi prediksi kelembaban (±{threshold}): {accuracy:.2f}")

Akurasi prediksi kelembaban (±5.0): 0.41


# Incremental Learning (stochastic gradient descent)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import time

# Target kota
target_column = 'Vancouver'

# Label klasifikasi: 1 jika kelembaban ≥ 70, else 0
df_hum['label'] = df_hum[target_column].apply(lambda x: 1 if x >= 70 else 0)

# Fitur: semua kota kecuali target dan datetime
# Check if 'datetime' column exists before dropping
columns_to_drop = [target_column, 'label']
if 'datetime' in df_hum.columns:
    columns_to_drop.insert(0, 'datetime')

X = df_hum.drop(columns=columns_to_drop).values
y = df_hum['label'].values

# Scaling fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Bagi data jadi train-test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Bagi train ke dalam batch (simulasi data stream)
batch_size = 1000
n_batches = len(X_train) // batch_size

# Inisialisasi model
model = SGDClassifier(loss="log_loss", random_state=42)

# Training incremental per batch
for i in range(n_batches):
    start = i * batch_size
    end = start + batch_size
    X_batch = X_train[start:end]
    y_batch = y_train[start:end]

    if i == 0:
        model.partial_fit(X_batch, y_batch, classes=np.array([0, 1]))
    else:
        model.partial_fit(X_batch, y_batch)

    print(f"Model trained on batch {i+1}/{n_batches}")
    time.sleep(0.1)  # Simulasi delay

# Evaluasi pada test set
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nFinal accuracy on test set: {acc:.2f}")

Model trained on batch 1/29
Model trained on batch 2/29
Model trained on batch 3/29
Model trained on batch 4/29
Model trained on batch 5/29
Model trained on batch 6/29
Model trained on batch 7/29
Model trained on batch 8/29
Model trained on batch 9/29
Model trained on batch 10/29
Model trained on batch 11/29
Model trained on batch 12/29
Model trained on batch 13/29
Model trained on batch 14/29
Model trained on batch 15/29
Model trained on batch 16/29
Model trained on batch 17/29
Model trained on batch 18/29
Model trained on batch 19/29
Model trained on batch 20/29
Model trained on batch 21/29
Model trained on batch 22/29
Model trained on batch 23/29
Model trained on batch 24/29
Model trained on batch 25/29
Model trained on batch 26/29
Model trained on batch 27/29
Model trained on batch 28/29
Model trained on batch 29/29

Final accuracy on test set: 0.82
